In [2]:
import pyodbc
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

True

In [3]:
# Settings
SERVER = os.environ.get("SQL_SERVER")
DATABASE = os.environ.get("SQL_DATABASE")
USERNAME = os.environ.get("SQL_USERNAME")
PASSWORD = os.environ.get("SQL_PASSWORD")

# Conexion
connectionString = f"Driver={{ODBC Driver 17 for SQL Server}};SERVER={SERVER},1433;DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}"


In [4]:
# Función de conexión para manejar errores de conexión
def connect_db():
    try:
        return pyodbc.connect(connectionString)
    except pyodbc.Error as e:
        print(f"Error al conectar a la base de datos: {e}")
        return None

## Cargar Tabla

In [5]:
def load_table_to_dataframe(table_name):
    """
    Carga los datos de una tabla desde la base de datos y los devuelve como un DataFrame de pandas.

    Args:
        table_name (str): El nombre de la tabla a consultar.

    Returns:
        pd.DataFrame | None: Un DataFrame con los datos de la tabla,
        o None si ocurre un error.
    """
    # Conectar a la base de datos
    conn = connect_db()
    if conn is None:
        print(
            "No se pudo establecer conexión con la base de datos para cargar la tabla."
        )
        return None

    try:
        cursor = conn.cursor()

        # Crear consulta SQL para seleccionar toda la tabla
        query = f"SELECT * FROM {table_name}"
        cursor.execute(query)
        rows = cursor.fetchall()
        columns = [column[0] for column in cursor.description]

        # Debugging: verificar datos recuperados
        print("Rows:", rows)
        print("Columns:", columns)

        # Limpiar los datos para manejar valores vacíos
        cleaned_rows = [
            tuple(value if value != "" else None for value in row) for row in rows
        ]

        # Convertir las filas en un DataFrame de pandas
        df = pd.DataFrame(cleaned_rows, columns=columns)
        print("Datos cargados exitosamente desde la tabla.")

        return df

    except Exception as e:
        print(f"Error al cargar los datos de la tabla '{table_name}': {e}")
        return None

    finally:
        if conn:
            conn.close()

In [11]:
# Cargamos tabla
ds_contacto_persona = load_table_to_dataframe("expo25_contacto_persona")
ds_contacto_persona

Rows: []
Columns: ['id_contacto', 'fecha_registro', 'nombre_contacto', 'nombre_empresa', 'mail_contacto', 'numero_contacto', 'motivo_contacto']
Datos cargados exitosamente desde la tabla.


,id_contacto,fecha_registro,nombre_contacto,nombre_empresa,mail_contacto,numero_contacto,motivo_contacto


In [7]:
ds_contacto_persona = load_table_to_dataframe("expo25_espacios")
ds_contacto_persona

Rows: [(4001, 'No', ''), (4002, 'No', ''), (4003, 'No', ''), (4004, 'No', '')]
Columns: ['id_espacio', 'ocupado', 'nombre_empresa']
Datos cargados exitosamente desde la tabla.


,id_espacio,ocupado,nombre_empresa
0,4001,No,None
1,4002,No,None
2,4003,No,None
3,4004,No,None


## Modificar Tabla

## Cambia Nombre, Crea Tabla, Copia datos, Verifica Datos, Elimina la Vieja

In [ ]:
def ejecutar_script_eliminar_identity():
    # Conexión a la base de datos
    try:
        conn = connect_db()
        cursor = conn.cursor()

        # Script SQL para eliminar la propiedad IDENTITY
        script_sql = """
                -- Paso 1: Renombrar la tabla original
        EXEC sp_rename 'expo25_espacios', 'expo25_espacios_old';

        -- Paso 2: Crear una nueva tabla sin la propiedad IDENTITY en la columna id_espacio
        CREATE TABLE expo25_espacios (
            id_espacio INT PRIMARY KEY, -- Sin IDENTITY
            ocupado NVARCHAR(3) NOT NULL CHECK (ocupado IN ('Si', 'No')),            
            nombre_empresa NVARCHAR(255) NULL
        );
        
          -- Paso 3: Copiar los datos de la tabla original a la nueva tabla
        INSERT INTO expo25_espacios (id_espacio, ocupado, id_empresa, nombre_empresa)
        SELECT id_espacio, ocupado, id_empresa, nombre_empresa
        FROM expo25_espacios_old;

        -- Paso 4: (Opcional) Verificar los datos en la nueva tabla
        SELECT * FROM expo25_espacios;
      


        -- Paso 5: Eliminar la tabla original si todo está correcto
        DROP TABLE expo25_espacios_old;
        """


        # Ejecutar el script SQL
        cursor.execute(script_sql)
        conn.commit()
        print("Script ejecutado con éxito. La columna IDENTITY fue eliminada.")

    except pyodbc.Error as e:
        print(f"Error al ejecutar el script: {e}")

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Llamar a la función para ejecutar el script





#EJECUTAR
## ejecutar_script_eliminar_identity()

Script ejecutado con éxito. La columna IDENTITY fue eliminada.


## Crear Tabla

In [ ]:

# Crear tabla QUIERO_ME_CONTACTEN
def create_table_expo25_contacto_persona():
    try:
        with connect_db() as conn:
            with conn.cursor() as cursor:
                cursor.execute(
                    """
                    CREATE TABLE expo25_contacto_persona (
                        id_contacto INT IDENTITY(1,1) PRIMARY KEY,      -- Identificador único del contacto
                        fecha_registro DATETIME DEFAULT GETDATE(),      -- Fecha de registro, valor por defecto la fecha actual
                        nombre_contacto NVARCHAR(100) NOT NULL,         -- Nombre de la persona a contactar
                        nombre_empresa NVARCHAR(100) NULL,             -- Nombre de la empresa (opcional)
                        mail_contacto NVARCHAR(150) NOT NULL,           -- Correo electrónico de contacto
                        numero_contacto NVARCHAR(15) NOT NULL,          -- Número de teléfono de contacto                        
                        motivo_contacto NVARCHAR(MAX) NOT NULL          -- Descripción del motivo de contacto
                    )
                    """
                )
                conn.commit()
                print("Tabla 'expo25_contacto_persona' creada exitosamente.")
    except pyodbc.Error as e:
        print(f"Error al crear la tabla 'expo25_contacto_persona': {e}")


In [14]:

# Crear tabla QUIERO_ME_CONTACTEN
def create_table_expo25_espacios():
    try:
        with connect_db() as conn:
            with conn.cursor() as cursor:
                cursor.execute(
                    """
                    CREATE TABLE expo25_espacios (
                    id_espacio INT PRIMARY KEY IDENTITY(1,1),
                    ocupado NVARCHAR(3) NOT NULL CHECK (ocupado IN ('Si', 'No')),
                    id_empresa UNIQUEIDENTIFIER NULL,
                    nombre_empresa NVARCHAR(255) NULL
                );
                    """
                )
                conn.commit()
                print("Tabla 'expo25_contacto_persona' creada exitosamente.")
    except pyodbc.Error as e:
        print(f"Error al crear la tabla 'expo25_contacto_persona': {e}")

create_table_expo25_espacios()

Tabla 'expo25_contacto_persona' creada exitosamente.
